In [32]:
# Import all necessary packages

import pandas as pd
import matplotlib
import seaborn as sns
import numpy as np
import psycopg2
import sqlalchemy

from sql_functions import get_dataframe
from sql_functions import get_engine
from sql_functions import get_sql_config



# Step 1: Import file and create dataframe with individual variable

In [33]:
# Import excel file for test
fat_supply_eu = pd.read_excel("data/Seafood_Consumption_data/fao/protein_fat_supply_eu_countries_yearly_fao.xlsx")

In [34]:
# Use first row as column names and delete first row
fat_supply_eu = fat_supply_eu.rename(columns=fat_supply_eu.iloc[0]).drop(fat_supply_eu.index[0])

# Step 2: Inspect dataframe with head(), info(), shape, columns, tail(), describe()

In [35]:
fat_supply_eu.info()
#fat_supply_eu.head()
#fat_supply_eu.shape
#fat_supply_eu.columns
#fat_supply_eu.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3360 entries, 1 to 3360
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Domain Code       3360 non-null   object
 1   Domain            3360 non-null   object
 2   Area Code (FAO)   3360 non-null   object
 3   Area              3360 non-null   object
 4   Element Code      3360 non-null   object
 5   Element           3360 non-null   object
 6   Item Code         3360 non-null   object
 7   Item              3360 non-null   object
 8   Year Code         3360 non-null   object
 9   Year              3360 non-null   object
 10  Unit              3240 non-null   object
 11  Value             3240 non-null   object
 12  Flag              3240 non-null   object
 13  Flag Description  3240 non-null   object
dtypes: object(14)
memory usage: 367.6+ KB


In [36]:
fat_supply_eu.head(100)

,Domain Code,Domain,Area Code (FAO),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
1,FBS,Food Balances (2010-),11.0,Austria,664.0,Food supply (kcal/capita/day),2761.0,Freshwater Fish,2010.0,2010.0,kcal/capita/day,7.0,Fc,Calculated data
2,FBS,Food Balances (2010-),11.0,Austria,664.0,Food supply (kcal/capita/day),2761.0,Freshwater Fish,2011.0,2011.0,kcal/capita/day,7.0,Fc,Calculated data
3,FBS,Food Balances (2010-),11.0,Austria,664.0,Food supply (kcal/capita/day),2761.0,Freshwater Fish,2012.0,2012.0,kcal/capita/day,7.0,Fc,Calculated data
4,FBS,Food Balances (2010-),11.0,Austria,664.0,Food supply (kcal/capita/day),2761.0,Freshwater Fish,2013.0,2013.0,kcal/capita/day,7.0,Fc,Calculated data
5,FBS,Food Balances (2010-),11.0,Austria,664.0,Food supply (kcal/capita/day),2761.0,Freshwater Fish,2014.0,2014.0,kcal/capita/day,7.0,Fc,Calculated data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,FBS,Food Balances (2010-),11.0,Austria,664.0,Food supply (kcal/capita/day),2765.0,Crustaceans,2015.0,2015.0,kcal/capita/day,2.0,Fc,Calculated data
97,FBS,Food Balances (2010-),11.0,Austria,664.0,Food supply (kcal/capita/day),2765.0,Crustaceans,2016.0,2016.0,kcal/capita/day,2.0,Fc,Calculated data
98,FBS,Food Balances (2010-),11.0,Austria,664.0,Food supply (kcal/capita/day),2765.0,Crustaceans,2017.0,2017.0,kcal/capita/day,2.0,Fc,Calculated data
99,FBS,Food Balances (2010-),11.0,Austria,664.0,Food supply (kcal/capita/day),2765.0,Crustaceans,2018.0,2018.0,kcal/capita/day,2.0,Fc,Calculated data


# Step 3: Delete non necessary columns and rows

In [37]:
# delete 'Domain Code', 'Domain', 'Area Code (FAO)', 'Element Code', 'Item Code', 'Year Code', 'Unit', 'Flag', 'Flag Description'
fat_supply_eu.drop(['Domain Code', 'Domain', 'Area Code (FAO)', 'Element Code', 'Item Code', 'Year Code', 'Unit', 'Flag', 'Flag Description'], axis=1, inplace=True)

In [38]:
fat_supply_eu.Item.unique()
#fat_supply_eu.head()

array(['Freshwater Fish', 'Pelagic Fish', 'Marine Fish, Other',
       'Crustaceans'], dtype=object)

# Step 4: Rename columns lower case, snake case, spaces, delimiters 

In [39]:
fat_supply_eu.columns = fat_supply_eu.columns.str.lower()
fat_supply_eu.columns = fat_supply_eu.columns.str.replace(' ','_')
fat_supply_eu.columns = fat_supply_eu.columns.str.strip()

# Step 5: Rename row value names. First letter upper case e.g. EUR = Eur

In [40]:
fat_supply_eu['area'] = fat_supply_eu['area'].str.capitalize()
fat_supply_eu['element'] = fat_supply_eu['element'].str.capitalize()
fat_supply_eu['item'] = fat_supply_eu['item'].str.capitalize()

In [41]:
fat_supply_eu['area'] = fat_supply_eu['area'].str.replace('Czechia','Czech Republic')
fat_supply_eu.area.unique()


array(['Austria', 'Belgium', 'Belgium-luxembourg', 'Bulgaria', 'Croatia',
       'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland',
       'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy',
       'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands',
       'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain',
       'Sweden'], dtype=object)

# Step 9: Change data types if necessary:   
* year = integer
* volume = float, round to one decimal. Convert kilograms into tonnes (1t = 1000 kg)
* value = float, round to two decimals

In [42]:
fat_supply_eu.year = fat_supply_eu.year.astype(int)
fat_supply_eu.value = fat_supply_eu.value.astype(float)

In [43]:
fat_supply_eu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3360 entries, 1 to 3360
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   area     3360 non-null   object 
 1   element  3360 non-null   object 
 2   item     3360 non-null   object 
 3   year     3360 non-null   int64  
 4   value    3240 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 131.4+ KB


In [44]:
# convert table from days fat supply to yearly fat supply
fat_supply_eu['element'] = fat_supply_eu['element'].str.replace('/day','/year')
fat_supply_eu['value'] = fat_supply_eu['value'].mul(365.25).round(2)

# Step 10: Species cleaning 

In [45]:
#Delete row where 'Item' is 'Freshwater fish', 'Marine fish other'
fat_supply_eu = fat_supply_eu[fat_supply_eu["item"].str.contains("Freshwater") == False]
fat_supply_eu = fat_supply_eu[fat_supply_eu["item"].str.contains("Marine") == False]

# Step 7: Inspect Null Values / NaNs and datatypes with info()

In [46]:
fat_supply_eu[fat_supply_eu['value'].isna()]
#there are no values for area 'Belgium-luxembourg'
# rows with are 'Belgium-luxembourg'are deleted
fat_supply_eu = fat_supply_eu[fat_supply_eu["area"].str.contains("Belgium-luxembourg") == False]

In [47]:
fat_supply_eu.rename(columns={'area': 'country'}, inplace=True)

In [48]:
fao_fat_supply_eu = fat_supply_eu

# Import files

In [49]:
# Import excel file for test
fao_seafood_capita = pd.read_excel("data/Seafood_Consumption_data/fao/seafood_per_captia_eu_countries_quantity_yearly_fao.xlsx")

fao_seafood_capita.rename(columns = {'category_species':'categories_species'}, inplace = True)
#Delete rows where 'categories_species' is 'Freshwater fish', 'Marine fish other'
fao_seafood_capita = fao_seafood_capita[fao_seafood_capita["categories_species"].str.contains("Freshwater") == False]
fao_seafood_capita = fao_seafood_capita[fao_seafood_capita["categories_species"].str.contains("Marine") == False]

# Convert values into floats
fao_seafood_capita = fao_seafood_capita.apply(pd.to_numeric, errors='ignore')

fao_seafood_capita['country'] = fao_seafood_capita['country'].str.capitalize()
fao_seafood_capita['categories_species'] = fao_seafood_capita['categories_species'].str.capitalize()
fao_seafood_capita['country'] = fao_seafood_capita['country'].str.replace('Czechia','Czech Republic')
fao_seafood_capita.country.unique()

#Put years in rows
fao_seafood_capita = fao_seafood_capita.melt(id_vars=["country", "categories_species"], # Spalten die gleich bleiben sollen
                                                           var_name="year", # neue column
                                                         value_name="volume_kg_capita") # welche Werte sollen zugeordnet werden

fao_seafood_capita['year'] = fao_seafood_capita['year'].astype(int)

In [50]:
# Import excel file for test
fao_seafood_eu = pd.read_excel("data/Seafood_Consumption_data/fao/seafood_as_food_eu_countries_quantity_yearly_fao.xlsx")

#Delete rows where 'categories_species' is 'Freshwater fish', 'Marine fish other'
fao_seafood_eu = fao_seafood_eu[fao_seafood_eu["categories_species"].str.contains("Freshwater") == False]
fao_seafood_eu = fao_seafood_eu[fao_seafood_eu["categories_species"].str.contains("Marine") == False]

# Convert values into floats
fao_seafood_eu = fao_seafood_eu.apply(pd.to_numeric, errors='ignore')

# multiply values with 1000
fao_seafood_eu.iloc[:,2:] = fao_seafood_eu.iloc[:,2:].mul(1000)

fao_seafood_eu['country'] = fao_seafood_eu['country'].str.capitalize()
fao_seafood_eu['categories_species'] = fao_seafood_eu['categories_species'].str.capitalize()

fao_seafood_eu['country'] = fao_seafood_eu['country'].str.replace('Czechia','Czech Republic')
fao_seafood_eu.country.unique()

fao_seafood_eu = fao_seafood_eu.melt(id_vars=["country", "categories_species"], # Spalten die gleich bleiben sollen
                                                           var_name="year", # neue column
                                                         value_name="volume_in_t") # welche Werte sollen zugeordnet werden

fao_seafood_eu['year'] = fao_seafood_eu['year'].astype(int)
fao_seafood_eu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             540 non-null    object 
 1   categories_species  540 non-null    object 
 2   year                540 non-null    int64  
 3   volume_in_t         540 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 17.0+ KB


# Merge all three fao-df

In [51]:
#first merge

fao_merge_eu = pd.merge(fao_seafood_eu, fao_seafood_capita,  how='left', left_on=['country','categories_species','year'], right_on = ['country','categories_species','year'])

In [52]:
fao_merge_eu.head()

,country,categories_species,year,volume_in_t,volume_kg_capita
0,Austria,Crustaceans,2010,10930.0,1.30
1,Austria,Pelagic fish,2010,32750.0,3.89
2,Belgium,Crustaceans,2010,36730.0,3.36
3,Belgium,Pelagic fish,2010,44330.0,4.05
4,Bulgaria,Crustaceans,2010,870.0,0.12


In [53]:
fao_fat_supply_eu.element.unique()

array(['Food supply (kcal/capita/year)',
       'Protein supply quantity (g/capita/year)',
       'Fat supply quantity (g/capita/year)'], dtype=object)

In [54]:
fao_fat_supply_eu.head()

,country,element,item,year,value
31,Austria,Food supply (kcal/capita/year),Pelagic fish,2010,3652.50
32,Austria,Food supply (kcal/capita/year),Pelagic fish,2011,4383.00
33,Austria,Food supply (kcal/capita/year),Pelagic fish,2012,4017.75
34,Austria,Food supply (kcal/capita/year),Pelagic fish,2013,4383.00
35,Austria,Food supply (kcal/capita/year),Pelagic fish,2014,3652.50


In [55]:
#Substrings for second merge
ff = fao_fat_supply_eu.query("element == 'Food supply (kcal/capita/year)'")
ff.rename(columns={'value': 'kcal/capita/year'}, inplace=True)
ff.rename(columns={'item': 'categories_species'}, inplace=True)
del ff['element']

pp = fao_fat_supply_eu.query("element == 'Protein supply quantity (g/capita/year)'")
pp.rename(columns={'value': 'protein_g/capita/year'}, inplace=True)
pp.rename(columns={'item': 'categories_species'}, inplace=True)
del pp['element']

fa = fao_fat_supply_eu.query("element == 'Fat supply quantity (g/capita/year)'")
fa.rename(columns={'value': 'fat_g/capita/year'}, inplace=True)
fa.rename(columns={'item': 'categories_species'}, inplace=True)
del fa['element']
ff.head(20)

/var/folders/nr/h230z7d96qv_m1lwv_pvcfsw0000gn/T/ipykernel_988/2583094817.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ff.rename(columns={'value': 'kcal/capita/year'}, inplace=True)
/var/folders/nr/h230z7d96qv_m1lwv_pvcfsw0000gn/T/ipykernel_988/2583094817.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ff.rename(columns={'item': 'categories_species'}, inplace=True)
/var/folders/nr/h230z7d96qv_m1lwv_pvcfsw0000gn/T/ipykernel_988/2583094817.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

,country,categories_species,year,kcal/capita/year
31,Austria,Pelagic fish,2010,3652.50
32,Austria,Pelagic fish,2011,4383.00
33,Austria,Pelagic fish,2012,4017.75
34,Austria,Pelagic fish,2013,4383.00
35,Austria,Pelagic fish,2014,3652.50
36,Austria,Pelagic fish,2015,3652.50
37,Austria,Pelagic fish,2016,3652.50
38,Austria,Pelagic fish,2017,3652.50
39,Austria,Pelagic fish,2018,3652.50
40,Austria,Pelagic fish,2019,3652.50


In [56]:
#second merge
fao_merge_eu = pd.merge(fao_merge_eu, ff,  how='left', left_on=['country','categories_species','year'], right_on = ['country','categories_species','year'])
fao_merge_eu = pd.merge(fao_merge_eu, pp,  how='left', left_on=['country','categories_species','year'], right_on = ['country','categories_species','year'])
fao_merge_eu = pd.merge(fao_merge_eu, fa,  how='left', left_on=['country','categories_species','year'], right_on = ['country','categories_species','year'])


In [57]:
fao_merge_eu.head()

,country,categories_species,year,volume_in_t,volume_kg_capita,kcal/capita/year,protein_g/capita/year,fat_g/capita/year
0,Austria,Crustaceans,2010,10930.0,1.30,365.25,102.27,3.65
1,Austria,Pelagic fish,2010,32750.0,3.89,3652.50,416.38,193.58
2,Belgium,Crustaceans,2010,36730.0,3.36,1461.00,299.50,18.26
3,Belgium,Pelagic fish,2010,44330.0,4.05,4383.00,529.61,226.46
4,Bulgaria,Crustaceans,2010,870.0,0.12,0.00,7.30,0.00


In [58]:
##fao_merge_eu

# Import get_engine from sql_functions.py. You will need to restart your kernel and rerun at this point since we changed the module since we first imported it.
from sql_functions import get_engine
# create a variable called engine using the get_engine function
engine = get_engine()

# Set the schema to your capstone schema and the table_name variable to 'fish_catch' etc + your initials/group number

schema = 'capstone_fish_are_friends' # capstone schema example capstone_fish_are_friends
table_name = 'fao_merge_eu' # Example: 'carriers_pw' for Philipp Wendt


# Write records stored in a dataframe to SQL database using to_sql() function
if engine!=None:
   try:
      fao_merge_eu.to_sql(name=table_name, # Name of SQL table variable
                        con=get_engine(), # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
      print(f"The {table_name} table was imported successfully.")
    # Error handling
   except (Exception, psycopg2.DatabaseError) as error:
      print(error)
      engine = None
else:
   print('No engine')

The fao_merge_eu table was imported successfully.
